eda1 link: https://github.com/fractaldatalearning/Capstone2/blob/main/notebooks/eda1_w_data_direct_from_wrangling.ipynb

In eda1 I was exploring data as-is right after basic wrangling/cleaning. I'll definitely need more rows and columns, and this notebook is for figuring out how to create new rows that indicate for each order where there's a non-order of a previously-ordered item. Starting with just one user. 

In [1]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.lines as mlines

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

from sklearn.tree import export_graphviz
from six import StringIO  
from IPython.display import Image  
import pydotplus

import itertools
from library.sb_utils import save_file
import json

In [2]:
df = pd.read_csv('../data/processed/full_data_cleaned.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33819106 entries, 0 to 33819105
Data columns (total 15 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_by_user_sequence  int64  
 3   order_dow               int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              int64  
 7   add_to_cart_sequence    int64  
 8   reordered               int64  
 9   product_name            object 
 10  aisle_name              object 
 11  dept_name               object 
 12  aisle_id                int64  
 13  department_id           int64  
 14  eval_set                object 
dtypes: float64(1), int64(10), object(4)
memory usage: 3.8+ GB


In [3]:
# Find just one user to explore with.
prolific_users = df[df['order_by_user_sequence']>100]
len(prolific_users['user_id'].unique())

0

In [4]:
prolific_users = df[df['order_by_user_sequence']==100]
len(prolific_users['user_id'].unique())

867

In [5]:
prolific_users['add_to_cart_sequence'].max()

47

In [6]:
prolific_user_w_biggest_cart = prolific_users[prolific_users['add_to_cart_sequence']==47]
prolific_user_w_biggest_cart

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_sequence,reordered,product_name,aisle_name,dept_name,aisle_id,department_id,eval_set
5263305,1629423,32099,100,1,14,1.0,27966,47,1,Organic Raspberries,packaged vegetables fruits,produce,123,4,train


In [7]:
practice_user = df[df['user_id']==32099]
practice_user.sample(7)

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_sequence,reordered,product_name,aisle_name,dept_name,aisle_id,department_id,eval_set
5262797,1482171,32099,70,4,11,2.0,41315,24,1,Organic Handmade Granola,granola,breakfast,57,14,prior
5263142,1567497,32099,91,0,16,2.0,23909,1,1,2% Reduced Fat Milk,milk,dairy eggs,84,16,prior
5261521,455159,32099,7,2,8,2.0,8580,7,1,VitaminWater Zero™ XXX Acai Blueberry Pomegranate,energy sports drinks,beverages,64,7,prior
5261437,2494101,32099,3,4,11,3.0,30702,31,0,Hampshire 100% Natural Grade A Pasteurized Sou...,other creams cheeses,dairy eggs,108,16,prior
5263013,3183255,32099,81,1,12,1.0,10893,7,0,Pure Vitamin C-1000 mg Caplets,vitamins supplements,personal care,47,11,prior
5262478,2301592,32099,52,4,10,6.0,45603,12,1,Trilogy Kombucha Drink,refrigerated,beverages,31,7,prior
5263118,2069065,32099,89,3,16,1.0,11408,15,0,Crushed Tomatoes,canned jarred vegetables,canned goods,81,15,prior


In [8]:
# Save version of this df before adding new rows
pu_unchanged = practice_user
pu_unchanged.head(2)

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_sequence,reordered,product_name,aisle_name,dept_name,aisle_id,department_id,eval_set
5261360,2959648,32099,1,0,17,NaN,40285,18,0,Traditional Snack Mix,trail mix snack mix,snacks,125,19,prior
5261361,2959648,32099,1,0,17,NaN,27966,15,0,Organic Raspberries,packaged vegetables fruits,produce,123,4,prior


In [9]:
# Tease out whether each item is reordered or not each order.
order1 = practice_user[practice_user['order_by_user_sequence']==1
                           ]['product_id'].unique().tolist()
len(order1)

26

In [10]:
order2 = practice_user[practice_user['order_by_user_sequence']==2
                           ]['product_id'].unique().tolist()
len(order2)

12

In [11]:
only1 = [x for x in order1 if x not in order2]
len (only1)

25

In [12]:
# Add rows so that non-reordered items are present in order 2
practice_user[practice_user['order_by_user_sequence']==1].sample(1)

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_sequence,reordered,product_name,aisle_name,dept_name,aisle_id,department_id,eval_set
5261371,2959648,32099,1,0,17,NaN,37940,14,0,Original Strawberry Low Fat Yogurt,yogurt,dairy eggs,120,16,prior


In [13]:
practice_user[practice_user['order_by_user_sequence']==2].sample(1)

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_sequence,reordered,product_name,aisle_name,dept_name,aisle_id,department_id,eval_set
5261393,309518,32099,2,1,15,1.0,19653,10,0,Rich Taste Extra Virgin Olive Oil,oils vinegars,pantry,19,13,prior


In [14]:
order2deets = {'order_id':309518, 'user_id':32099, 
               'order_by_user_sequence':2, 'order_dow':1, 
               'order_hour_of_day':15, 'days_since_prior_order': 1.0,
               'product_id': only1, 'reordered':0}

order2_new_rows = pd.DataFrame.from_dict(order2deets)

order2_new_rows.head()

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered
0,309518,32099,2,1,15,1.0,40285,0
1,309518,32099,2,1,15,1.0,27966,0
2,309518,32099,2,1,15,1.0,34969,0
3,309518,32099,2,1,15,1.0,7419,0
4,309518,32099,2,1,15,1.0,26209,0


In [15]:
practice_user = pd.concat([practice_user, order2_new_rows])
practice_user.sample(10)

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_sequence,reordered,product_name,aisle_name,dept_name,aisle_id,department_id,eval_set
5262725,1450148,32099,66,3,9,2.0,12916,21.0,0,Ginger Ale,soft drinks,beverages,77.0,7.0,prior
5262994,2312765,32099,80,0,12,3.0,29103,6.0,1,Zero Go-Go Mixed Berry Vitamin Water,energy sports drinks,beverages,64.0,7.0,prior
5261599,2819872,32099,10,0,14,3.0,11576,7.0,1,Corn Chips,chips pretzels,snacks,107.0,19.0,prior
5261529,455159,32099,7,2,8,2.0,41387,12.0,0,Plastic Spoons,plates bowls cups flatware,household,111.0,17.0,prior
5263334,1629423,32099,100,1,14,1.0,44051,8.0,1,Zero Vitamin Water,energy sports drinks,beverages,64.0,7.0,train
5262397,721754,32099,47,0,15,3.0,33827,12.0,1,Fat Free French Vanilla Creamer,cream,dairy eggs,53.0,16.0,prior
5261910,3351094,32099,23,6,10,1.0,46041,1.0,0,Beef Franks,hot dogs bacon sausage,meat seafood,106.0,12.0,prior
5262934,688733,32099,77,1,14,2.0,2340,2.0,0,Beef Stew Spices & Seasonings,spices seasonings,pantry,104.0,13.0,prior
5261957,3380792,32099,24,6,12,7.0,30702,32.0,1,Hampshire 100% Natural Grade A Pasteurized Sou...,other creams cheeses,dairy eggs,108.0,16.0,prior
5262949,2656090,32099,78,3,11,2.0,44051,14.0,1,Zero Vitamin Water,energy sports drinks,beverages,64.0,7.0,prior


Now find a way to iterate and do this for every order. Find items from order n not reordered in order n+1. Find deets for order n+1. Add rows to practice_user with n+1 deets and n-only products

In [16]:
# This person has 100 orders, so I'm working from n (order_by_user_sequence) 2-99 and n+1 would 
# be orders 3-100

# Start with creating orders' deets in dictionary form

orders_deets = practice_user.iloc[:,0:6].reset_index(drop=True)
orders_deets = orders_deets.groupby('order_by_user_sequence').first()
orders_deets['sequence_column'] = orders_deets.index
orders_deets = orders_deets.reset_index(drop=True)
orders_deets.index = orders_deets.index + 1
orders_deets = orders_deets.rename(columns={'sequence_column':
                                            'order_by_user_sequence'})
orders_deets = orders_deets[['order_id', 'user_id', 
                             'order_by_user_sequence', 'order_dow', 
                             'order_hour_of_day', 'days_since_prior_order']]
orders_deets = orders_deets.to_dict(orient='index')
orders_deets[0:6]

TypeError: unhashable type: 'slice'

In [ ]:
# Already finished with adding rows to order 2, can drop order 1.

orders_deets.pop(1)

In [ ]:
orders_deets[0:3]

In [ ]:
# Now iterate over practice_user df to find un-reordered items for each order and add them as 
# new rows.

for n in range(2,100):
    # Get items from order n not reordered in order n+1
    order_n = practice_user[practice_user['order_by_user_sequence']==n
                           ]['product_id'].unique().tolist()
    order_n1 = practice_user[practice_user['order_by_user_sequence']==(
        n+1)]['product_id'].unique().tolist()
    only_n = [x for x in order_n if x not in order_n1]
    # Get n1 deets from the big deets dict
    order_n1_deets = orders_deets.get(n+1)
    # Add to n1 deets dict with product ids from order_n
    order_n1_deets.update({'product_id': only_n})
    # Turn dict into df of new rows
    order_n1_new_rows = pd.DataFrame.from_dict(order_n1_deets)
    # Add new rows to practice_user df
    practice_user = pd.concat([practice_user, order_n1_new_rows])

practice_user.info()

In [ ]:
practice_user[practice_user['order_by_user_sequence']==3]

In [ ]:
# Deal with all the NaNs I just made. add_to_cart_sequence can stay: NaN is accurate in this 
# context. All new rows can have 'reordered' value of 0

practice_user['reordered'] = practice_user['reordered'].fillna(0)
practice_user.info()

In [ ]:
# Remaining rows can be filled based on the product_id column value Need to get dictionaries for 
# department_id:dept_name, aisle_id:aisle_name, product_id:product_name, 
# aisle_id:department_id, product_id:aisle_id

prod_id_name_dict = df.set_index('product_id').to_dict()['product_name']
prod_id_name_dict[0:6]

In [ ]:
aisle_id_name_dict = df.set_index('aisle_id').to_dict()['aisle_name']
aisle_id_name_dict[0:6]

In [ ]:
dept_id_name_dict = df.set_index('department_id').to_dict()['dept_name']
dept_id_name_dict

In [ ]:
prod_aisle_dict = df.set_index('product_id').to_dict()['aisle_id']
prod_aisle_dict[0:6]

In [ ]:
aisle_dept_dict = df.set_index('aisle_id').to_dict()['department_id']
aisle_dept_dict[0:6]

In [ ]:
practice_user['product_name'] = practice_user['product_name'].fillna(practice_user[
    'product_id'].map(prod_id_name_dict))

practice_user['aisle_id'] = practice_user['aisle_id'].fillna(practice_user['product_id'].map(
    prod_aisle_dict))

practice_user['department_id'] = practice_user['department_id'].fillna(practice_user[
    'aisle_id'].map(aisle_dept_dict))

practice_user['aisle_name'] = practice_user['aisle_name'].fillna(practice_user['aisle_id'].map(
    aisle_id_name_dict))

practice_user['dept_name'] = practice_user['dept_name'].fillna(practice_user[
    'department_id'].map(dept_id_name_dict))

practice_user.info()

The "eval set" column is a hold-over from this being a kaggle competition set. But it does indicate the length of the df for this user before I added all the rows. They had about 2k rows originally and now have 34k! Even if other users won't grow as much (this is the largest user) I could expect the full df to have hundreds of millions of rows. Probably best to find chunks of users to work with when modeling. 

In [ ]:
# Visualize reorder patterns for this user. Use the pu_unchanged dataset because I want to 
# compare reorders with all ordered items, not with all items ever ordered compounding in each 
# order.

user_reordered = pu_unchanged[pu_unchanged['reordered']==1]

plt.hist(pu_unchanged['order_dow'], color='blue', density=True, alpha=0.5, bins=7)
plt.hist(user_reordered['order_dow'], color='orange', density=True, alpha=0.5, bins=7)
plt.title('Proportion of Orders per Day of Week')
plt.xlabel('day of week')
plt.ylabel('portion')
blue = mlines.Line2D([], [], color='blue', marker='s', linestyle='None', markersize=10, 
                       label='all orders')
orange = mlines.Line2D([], [], color='orange', marker='s', linestyle='None', markersize=10, 
                       label='reorders')
plt.legend(handles=[blue, orange]);

In [ ]:
# In the overall df, dow changed little between reorders vs all orders This person seems similar. 

plt.hist(pu_unchanged['order_hour_of_day'], color='blue', density=True, alpha=0.5, bins=12)
plt.hist(user_reordered['order_hour_of_day'], color='orange', density=True, alpha=0.5, bins=12);
plt.title('Proportion of Orders by Hour of Day')
plt.xlabel('hour')
plt.ylabel('portion')
blue = mlines.Line2D([], [], color='blue', marker='s', linestyle='None', markersize=10, 
                       label='all orders')
orange = mlines.Line2D([], [], color='orange', marker='s', linestyle='None', markersize=10, 
                       label='reorders')
plt.legend(handles=[blue, orange]);

In [ ]:
# In overall df, most people do most reordering from 7-10am This person has less of a pattern

plt.hist(pu_unchanged['days_since_prior_order'], color='blue', density=True, alpha=0.5, bins=15)
plt.hist(user_reordered['days_since_prior_order'], color='orange', density=True, alpha=0.5, 
         bins=15)
plt.title('Proportion of Orders with Days Since Prior Order')
plt.xlabel('days')
plt.ylabel('portion')
blue = mlines.Line2D([], [], color='blue', marker='s', linestyle='None', markersize=10, 
                       label='all orders')
orange = mlines.Line2D([], [], color='orange', marker='s', linestyle='None', markersize=10, 
                       label='reorders')
plt.legend(handles=[blue, orange]);

This person only has 15 days between orders at most, which makes sense because they're a major customer. Overall, it seemed more reorders happened within 10 days. This person seems not to reorder in the first 4 days, which makes a sort of sense (no need to reorder if you just got something, no time to have run out). And they include reorders at a relatively high rate even if it has been more than 10 days since their last order.  

In [ ]:
pu_unchanged['add_to_cart_sequence'].max()

In [ ]:
plt.hist(pu_unchanged['add_to_cart_sequence'], color='blue', density=True, alpha=0.5, bins=29)
plt.hist(user_reordered['add_to_cart_sequence'], color='orange', density=True, alpha=0.5, 
         bins=47)
plt.title('In What Order are Reorders Placed in Basket?')
plt.xlabel('sequence added to basket')
plt.ylabel('portion')
blue = mlines.Line2D([], [], color='blue', marker='s', linestyle='None', markersize=10, 
                       label='all orders')
orange = mlines.Line2D([], [], color='orange', marker='s', linestyle='None', markersize=10, 
                       label='reorders')
plt.legend(handles=[blue, orange]);

Most people put reorders in their basket first. This person seems to have a regular pattern of placing some new items in the basket first, following by reorders, then more new, then more reorders, etc. 

In [ ]:
# Explore more of the order sequence.
plt.hist(pu_unchanged['order_by_user_sequence'], color='blue', density=True, alpha=0.5, bins=20)
plt.hist(user_reordered['order_by_user_sequence'], color='orange', density=True, alpha=0.5, 
         bins=20)
plt.title('Reorders by Order Position in Order Sequence')
plt.xlabel('order sequence number')
plt.ylabel('portion')
blue = mlines.Line2D([], [], color='blue', marker='s', linestyle='None', markersize=10, 
                       label='all orders')
orange = mlines.Line2D([], [], color='orange', marker='s', linestyle='None', markersize=10, 
                       label='reorders')
plt.legend(handles=[blue, orange]);

There is a similar trend here to the full df, with more reorders happening after more orders have been placed. But this person seems to have waited a bit longer before doing the bulk of their reordering. This person's behavior isn't grossly divergent from that of all users, though there are some differences. I'd like to do more exploration with this single user, and if I end up working with chunks of multiple users but not the entire df, I should make sure to try to find users whose behaviors reflect normal behavior, by choosing the users randomly.

In [ ]:
# Use a DecisionTree to visualize one way how all this *could* be modeled.
# Get only the potentially useful, non-redundant columns. 

practice_user.columns

In [ ]:
pu_4models = practice_user.loc[:,['order_id', 'user_id', 'order_by_user_sequence', 
                            'order_dow', 'order_hour_of_day', 'days_since_prior_order', 
                            'product_id','add_to_cart_sequence', 'reordered', 'aisle_id',
                           'department_id']]
pu_4models.head(2)

In [ ]:
# Deal with remaining NaN values
pu_4models.isnull().any()

In [ ]:
pu_4models['days_since_prior_order'].sort_values().unique()

In [ ]:
# Not sure if this is the best option eventually, but for now I'm just trying to get every 
# field to be a number. Having a negative number in a place when the value is truly nan makes 
# some sort of sense. 

pu_4models['days_since_prior_order'] = pu_4models[
    'days_since_prior_order'].fillna(-1)
pu_4models['days_since_prior_order'].sort_values().unique()

In [ ]:
pu_4models['add_to_cart_sequence'].sort_values().unique()

In [ ]:
# For add_to_cart_sequence, since no items originally had a value of 0, it seems to make sense to
# give the value of 0 to items that are never actually added to the cart. 

pu_4models['add_to_cart_sequence'] = pu_4models['add_to_cart_sequence'].fillna(0)
pu_4models['add_to_cart_sequence'].sort_values().unique()

In [ ]:
pu_4models.isnull().any()

In [ ]:
# Get ind/dep variables.

y = pu_4models['reordered']
X = pu_4models.drop('reordered', axis=1)

X.head(2)

In [ ]:
y = y.array.reshape(-1,1)

In [ ]:
# Model (expect massive problems from a modeling perspective, just trying to get a visual)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3)

clf = DecisionTreeClassifier(max_depth=5)
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.confusion_matrix(y_test, y_pred)

In [ ]:
# Visualize tree

dot_data = StringIO()
export_graphviz(clf, out_file=dot_data, filled=True, rounded=True, 
                special_characters=True, feature_names = X.columns, 
                class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('pu_4models.png')
Image(graph.create_png())

It looks like the order_by_user_sequence and add_to_cart_sequence are the features being used most to make decisions. And, for example, most samples classified as non-reorders occur with order_by_user_sequence <3.5. Plenty here is consistent with what the histograms showed. There are also tons of problems with this model, for example, unbalanced classes and categories being treated as ordinal just because their ids are integers. But it's been helpful in simply visualizing how a model could potentially shake out given existing data. 

In [ ]:
# Save all the dictionaries in files that can be imported later Because they take a long time to
# load and re-make themselves. 

with open('../data/processed/dicts/prod_id_name_dict.txt', 'w') as convert_file:
     convert_file.write(json.dumps(prod_id_name_dict))

In [ ]:
with open('../data/processed/dicts/aisle_id_name_dict.txt', 'w') as convert_file:
     convert_file.write(json.dumps(aisle_id_name_dict))

In [ ]:
with open('../data/processed/dicts/dept_id_name_dict.txt', 'w') as convert_file:
     convert_file.write(json.dumps(dept_id_name_dict))

In [ ]:
with open('../data/processed/dicts/prod_aisle_dict.txt', 'w') as convert_file:
     convert_file.write(json.dumps(prod_aisle_dict))

In [ ]:
with open('../data/processed/dicts/aisle_dept_dict.txt', 'w') as convert_file:
     convert_file.write(json.dumps(aisle_dept_dict))

In [ ]:
# And save the practice_user df to do more actual modeling
# with them in the next notebook. 

datapath = '../data/processed'
save_file(practice_user, 'practice_user.csv', datapath)

Next:
- Features need to be engineered with dummies and such to be more appropriate for modeling. Do that with the single user as the final step of EDA.
- I'll need to be working with some portion (which, I need to decide) of the original, full df. Generalize the row-creation procedure I used here on a dataset with more users, as transition into the preprocessing step. 

Note: I Attempted clustering products into sub-aisles, but this proved fairly fruitless and seems not to be worth the work/won't greatly improve predictions as far as I can tell. Throwaway notebook where I worked that out: https://github.com/fractaldatalearning/Capstone2/blob/main/notebooks/eda_trash_product_clusters.ipynb
- An alternative option would be to analyze product names and make some columns out of them: i.e. Organic column with 0/1 if "organic" appears in the product name. Do this as part of feature engineering in the next notebook.

Link to eda3 notebook for more work with this single user: https://github.com/fractaldatalearning/Capstone2/blob/main/notebooks/eda3_1user_modeling.ipynb